# Attacker model

Here we will put it all togheter using the generated synthetic data from `data_synthesis_playground.ipynb`

In [1]:
from mblearn import AttackModels, ShadowModels

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

First we need to make a target model (we will use RandomForest with 100 estimators)

## Target model

In [3]:
rf_target = RandomForestClassifier(n_estimators=100)
data, target = load_wine(return_X_y=True)



In [4]:
scaler = MinMaxScaler()
data_std = scaler.fit_transform(data)

# split to test membership in X_train
X_train, X_test, y_train, y_test = train_test_split(data_std, target, test_size=0.2)

rf_target.fit(X_train, y_train);

## Shadow model

Now train the Shadow models with synthetic data and the same learner (that's cheating a bit)

In [5]:
synth_data = pd.read_csv('synthetic_data.csv')
synth_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,label
0,0.472479,0.802700,0.681440,0.418567,0.397988,0.763361,0.491796,0.423902,0.511174,0.671752,0.358635,0.973335,0.600171,0
1,0.890156,0.656366,0.389551,0.511145,0.491457,0.589019,0.596019,0.171550,0.052004,0.390024,0.522649,0.697107,0.438737,0
2,0.640766,0.087517,0.956563,0.137550,0.698057,0.201077,0.999131,0.678409,0.390072,0.796449,0.337422,0.765274,0.946437,0
3,0.525594,0.557560,0.428098,0.310923,0.401149,0.288159,0.962907,0.870142,0.404686,0.477274,0.783790,0.996116,0.468679,0
4,0.943780,0.413054,0.972705,0.473483,0.979340,0.968235,0.563845,0.711525,0.532065,0.210514,0.372955,0.483156,0.594460,0


In [6]:
rf_shadow =  RandomForestClassifier(n_estimators=100)

In [7]:
sh = ShadowModels(5, synth_data, 3, rf_shadow)

In [8]:
shadow_data = sh.results

## Attacker

Now that we have the shadow dataset we can train the attacker model. The attacker learner doesn't need to be the same as the target so pick the one that performs the best.

`AttackModels` trains a model for each original class in shadow data (and in target model) with the in/out of training label as the target label.

In [9]:
rf_attack = RandomForestClassifier(n_estimators=100)

In [10]:
attacker = AttackModels(target_classes=3, attack_learner=rf_attack)

In [11]:
attacker.fit(shadow_data)

now lets test with all `X_train` and `X_test`

In [12]:
X_in = rf_target.predict_proba(X_train)

res_in = attacker.predict(X_in, y_train, batch=True)

In [13]:
X_out = rf_target.predict_proba(X_test)

res_out = attacker.predict(X_out, y_test, batch=True)

In [20]:
np.sum(np.argmax(res_in, axis=1)) / len(res_in)

1.0

In [21]:
np.sum(np.argmax(res_out, axis=1)) / len(res_out)

0.8888888888888888

Weeeel that doesn't looks like it is working correctly... but it is more confident with the data that was in the training of the target Model which is something :_)